# Аналитика ДТП по предоставленным данным

Заказчиком (Оунером задачи) выступает проект «Карта ДТП» https://dtp-stat.ru/ — некоммерческий проект, посвященный проблеме дорожно-транспортных происшествий в России. Это платформа сбора данных о ДТП, бесплатный и открытый сервис аналитики ДТП.

**Цель проекта** - провести глубокий анализ данных, сформулировать и проверить гипотезы, опираясь на доступные признаки.

**Задачи**
* Провести исследовательский анализ данных о ДТП за 2025 год.
* Сформулировать и проверить не менее трех гипотез, основываясь на имеющихся признаках.
* Провести анализ ДТП по всем датасетам, представленным на сайте.
* Построить дашборд используя любой удобный инструмент, с учетом того, что дашборд может быть опубликован

**Декомпозиция работы:**
* Загрузка датасета и знакомство с данными
* Предобработка данных
* Исследовательский анализ данных
* Проверка гипотез
* Общие выводы
* Подготовка дашборда

## 1. Загрузка датасета и знакомство с данными

Загрузим необходимые библиотеки:

In [8]:
#!pip install geopandas

import geopandas as gpd
import pandas as pd

# Настройка отображения Jupiter
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # вывод результатов без сокращения количества столбцов

Загрузим датасет:

In [15]:
gdf = gpd.read_file('krasnoiarskii-krai.geojson')

Skipping field tags: unsupported OGR type: 5
Skipping field nearby: unsupported OGR type: 5
Skipping field weather: unsupported OGR type: 5
Skipping field road_conditions: unsupported OGR type: 5
Skipping field participant_categories: unsupported OGR type: 5


Преобразуем датасет в DataFrame Pandas:

In [28]:
data = pd.DataFrame(gdf)

Откроем датасет и посмотрим на данные:

In [16]:
data.head(5)

,id,light,point,region,scheme,address,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,participants_count,geometry
0,228398,"В темное время суток, освещение включено","{ ""lat"": 53.651899999999998, ""long"": 91.573099...",Минусинский район,None,"д Солдатово, ул Некрасова, 26",Наезд на пешехода,2016-04-08 19:40:00,Легкий,"[ { ""year"": 2004, ""brand"": ""ВАЗ"", ""color"": ""Си...",0,"[ { ""role"": ""Пешеход"", ""gender"": ""Мужской"", ""v...",2,Красноярский край,3,POINT (91.5731 53.6519)
1,228419,"В темное время суток, освещение включено","{ ""lat"": 53.591099999999997, ""long"": 92.415599...",Минусинский район,220,"с Тигрицкое, ул Зеленая, 19-1",Столкновение,2016-04-08 00:10:00,Тяжёлый,"[ { ""year"": 1988, ""brand"": ""Прочие марки мотоц...",0,[ ],1,Красноярский край,2,POINT (92.4156 53.5911)
2,226939,Светлое время суток,"{ ""lat"": 53.784477000000003, ""long"": 92.198639 }",Минусинский район,200,"Минусинск-Курагино-Артемовск, 35 км",Столкновение,2019-12-31 09:15:00,С погибшими,"[ { ""year"": 1990, ""brand"": ""TOYOTA"", ""color"": ...",3,[ ],0,Красноярский край,3,POINT (92.19864 53.78448)
3,226971,"В темное время суток, освещение отсутствует","{ ""lat"": 53.678899999999999, ""long"": 91.716899...",Минусинский район,300,Р-257 Енисей Красноярск - Абакан - Кызыл - гра...,Столкновение,2016-12-27 08:10:00,Тяжёлый,"[ { ""year"": 2012, ""brand"": ""УАЗ"", ""color"": ""Мн...",0,[ ],3,Красноярский край,5,POINT (91.7169 53.6789)
4,226982,Светлое время суток,"{ ""lat"": 53.658299999999997, ""long"": 91.760300...",Минусинский район,610,Р-257 Енисей Красноярск - Абакан - Кызыл - гра...,Наезд на препятствие,2016-12-19 11:10:00,Тяжёлый,"[ { ""year"": 2012, ""brand"": ""ВАЗ"", ""color"": ""Мн...",0,[ ],1,Красноярский край,2,POINT (91.7603 53.6583)


Посмотрим общую информацию о датасете:

In [17]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35121 entries, 0 to 35120
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  35121 non-null  int32         
 1   light               35121 non-null  object        
 2   point               35121 non-null  object        
 3   region              35121 non-null  object        
 4   scheme              29290 non-null  object        
 5   address             33224 non-null  object        
 6   category            35121 non-null  object        
 7   datetime            35121 non-null  datetime64[ms]
 8   severity            35121 non-null  object        
 9   vehicles            35121 non-null  object        
 10  dead_count          35121 non-null  int32         
 11  participants        35121 non-null  object        
 12  injured_count       35121 non-null  int32         
 13  parent_region       35121 non-null  ob

Исходный датасет содержит 35121 запись о ДТП с 16 признаками:

* `id`: идентификатор
* `light`: время суток
* `point`: координаты (гео)
* `nearby`: комментарий по месту происшествия (где произошло)
* `region`: город/район
* `address`: адрес
* `weather`: погода
* `category`: тип ДТП
* `datetime`: дата и время происшествия
* `severity`: тяжесть ДТП/вред здоровью
* `vehicles`: участники – транспортные средства:
    * `year`: год производства транспортного средства
    * `brand`: марка транспортного средства
    * `color`: цвет транспортного средства
    * `model`: модель транспортного средства
    * `category`: категория транспортного средства
    * `role`: роль участника
    * `gender`: пол участника
    * `violations`: нарушения правил участником
    * `health_status`: состояние здоровья участника
    * `years_of_driving_experience`: стаж вождения участника (только у водителей)
* `dead_count`: кол-во погибших в ДТП
* `participants`: участники без транспортных средств (описание, как у участников внутри транспортных средств)
* `injured_count`: кол-во раненых в ДТП
* `parent_region`: родительский регион
* `road_conditions`: состояние дорожного покрытия
* `participants_count`: кол-во участников ДТП
* `participant_categories`: категории участников

## 2. Предобработка данных

Задачи данного раздела:

* Привести типы данных в соответствие
* Проверить пропуски в датасете и обработать их
* Проверить наличие явных и неявных дубликатов и обработать их
* По возможности дообогатить данные на основе имеющихся

### 2.1. Обработка пропусков данных

Проверим наличие NAN и NULL в датасете:

In [18]:
print(20*'==')
print('NULLS')
print(data.isnull().sum())
print(20*'==')
print('NANs')
print(data.isna().sum())
print(20*'==')

NULLS
id                       0
light                    0
point                    0
region                   0
scheme                5831
address               1897
category                 0
datetime                 0
severity                 0
vehicles                 0
dead_count               0
participants             0
injured_count            0
parent_region            0
participants_count       0
geometry                 2
dtype: int64
NANs
id                       0
light                    0
point                    0
region                   0
scheme                5831
address               1897
category                 0
datetime                 0
severity                 0
vehicles                 0
dead_count               0
participants             0
injured_count            0
parent_region            0
participants_count       0
geometry                 2
dtype: int64


Обнаружены пропуски в трех полях - `scheme` (данное поле не указано в описании данных), `address`, `geometry`.

Проверим какие данные содержатся в поле `scheme`:

In [19]:
data['scheme'].unique()

array([None, '220', '200', '300', '610', '600', '860', '910', '070',
       '810', '950', '430', '900', '500', '830', '030', '140', '850',
       '210', '040', '880', '400', '820', '940', '930', '190', '110',
       '130', '120', '840', '420', '100', '090', '060', '870', '050',
       '010', '800', '630', '920', '780', '760', '740', '620', '330',
       '960', '020', '980', '720', '410', '310', '770', '700', '750',
       '230', '730', '710', '970', '340', '440', '320'], dtype=object)

Здесь в основном представлены трехзначные числа в формате строки. Заполним пропуски заглушкой "unknown"

In [21]:
data['scheme'] = data['scheme'].fillna('unknown')

In [22]:
data['scheme'].unique()

array(['unknown', '220', '200', '300', '610', '600', '860', '910', '070',
       '810', '950', '430', '900', '500', '830', '030', '140', '850',
       '210', '040', '880', '400', '820', '940', '930', '190', '110',
       '130', '120', '840', '420', '100', '090', '060', '870', '050',
       '010', '800', '630', '920', '780', '760', '740', '620', '330',
       '960', '020', '980', '720', '410', '310', '770', '700', '750',
       '230', '730', '710', '970', '340', '440', '320'], dtype=object)

Пропуски по полю `address` также заполним заглушкой:

In [25]:
data['address'] = data['address'].fillna('unknown')

Проверим пропуски по полю `geometry`:

In [29]:
data[data['geometry'].isna()]

,id,light,point,region,scheme,address,category,datetime,severity,vehicles,dead_count,participants,injured_count,parent_region,participants_count,geometry
5127,235251,"В темное время суток, освещение не включено","{ ""lat"": null, ""long"": null }",Уярский район,830,"с Сушиновка, ул Лесная, 20",Наезд на пешехода,2017-10-14 18:50:00,Легкий,"[ { ""year"": 2001, ""brand"": ""TOYOTA"", ""color"": ...",0,"[ { ""role"": ""Пешеход"", ""gender"": ""Женский"", ""v...",1,Красноярский край,2,None
8251,240683,Светлое время суток,"{ ""lat"": null, ""long"": null }",Козульский район,200,Р-255 Сибирь Новосибирск - Кемерово - Краснояр...,Столкновение,2016-11-26 12:25:00,Легкий,"[ { ""year"": 2002, ""brand"": ""NISSAN"", ""color"": ...",0,[ ],1,Красноярский край,2,None


Точки нигде не указаны, удалим данные строки.